<a href="https://colab.research.google.com/github/ErickMartinezz/AprendizajeAutomatico2/blob/main/EleccionDeAutomoviles_PP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 30px;
    background: rgba(0, 0, 30, 0.8);
    border-radius: 20px;
    box-shadow: 0 0 40px #ff0000;
    width: 90%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h1 style="
        text-align: center;
        font-size: 32px;
        color: #ff0000;
        text-shadow: 0 0 15px #ff0000;
    ">🚗 Aplicación Avanzada: Car Evaluation</h1>
    <p style="font-size:18px; line-height:1.6;">
        En este proyecto analizaremos el dataset <b>Car Evaluation</b> utilizando técnicas de clustering y clasificación:
        <b>KMeans</b>, <b>KNN</b>, <b>RandomForest</b> y <b>Stacking Classifier</b>.
    </p>
    <p style="font-size:18px; line-height:1.6;">
        <b>Dataset elegido:</b>
        <a href="http://archive.ics.uci.edu/dataset/19/car+evaluation" style="color:#00ffff;">Car Evaluation (UCI)</a>
    </p>
</div>
""")


In [32]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 25px;
    background: rgba(0, 0, 30, 0.8);
    border-radius: 20px;
    box-shadow: 0 0 30px #00ffff;
    width: 80%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h2 style="
        text-align: center;
        font-size: 28px;
        color: #00ffff;
        text-shadow: 0 0 10px #00ffff;
    ">📋 Información Académica</h2>
    <p style="font-size:18px; line-height:1.6;">
        <b>Nombre Alumno:</b> Erick Martinez
    </p>
    <p style="font-size:18px; line-height:1.6;">
        <b>Docente:</b> Julio Paredes
    </p>
</div>
""")


In [33]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 30px;
    background: rgba(0, 0, 30, 0.85);
    border-radius: 20px;
    box-shadow: 0 0 35px #ff6600;
    width: 85%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h2 style="
        text-align: center;
        font-size: 28px;
        color: #ff6600;
        text-shadow: 0 0 12px #ff6600;
    ">🎯 Objetivo del Proyecto</h2>
    <p style="font-size:18px; line-height:1.6;">
        Clasificar autos según la variable <b>clase</b> (inaceptable, aceptable, bueno, muy bueno) usando:
    </p>
    <ul style="font-size:18px; line-height:1.6;">
        <li><b>KNN</b> (clasificador supervisado)</li>
        <li><b>Random Forest</b> (clasificador supervisado)</li>
        <li><b>StackingClassifier</b> (apilado de modelos)</li>
    </ul>
    <p style="font-size:18px; line-height:1.6;">
        Además, se calculará una valorización económica de las predicciones (<b>payoff matrix</b>) para comparar modelos según valor, no solo métricas.
    </p>
</div>
""")


#Librerias, impotacion de dataset


In [34]:
# @title
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')


np.random.seed(42)

In [35]:
# @title
!pip install ucimlrepo

In [36]:
# @title
from ucimlrepo import fetch_ucirepo

# fetch dataset
car_evaluation = fetch_ucirepo(id=19)

# data (as pandas dataframes)
X = car_evaluation.data.features
y = car_evaluation.data.targets

# metadata
print(car_evaluation.metadata)

# variable information
print(car_evaluation.variables)


{'uci_id': 19, 'name': 'Car Evaluation', 'repository_url': 'https://archive.ics.uci.edu/dataset/19/car+evaluation', 'data_url': 'https://archive.ics.uci.edu/static/public/19/data.csv', 'abstract': 'Derived from simple hierarchical decision model, this database may be useful for testing constructive induction and structure discovery methods.', 'area': 'Other', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1728, 'num_features': 6, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1988, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5JP48', 'creators': ['Marko Bohanec'], 'intro_paper': {'ID': 249, 'type': 'NATIVE', 'title': 'Knowledge acquisition and explanation for multi-attribute decision making', 'authors': 'M. Bohanec, V. Rajkovič', 'venue': '8th Intl Workshop on Expert Systems and their Applications, 

#EDA

In [48]:
# @title
print('\n--- Información general ---')
print(df.info())


print('\n--- Descripción rápida ---')
display(df.describe(include='all').T)


print('\n--- Distribución de la variable objetivo \"class\" ---')
display(df['class'].value_counts())


--- Información general ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   precio_compra         1728 non-null   object
 1   precio_mantenimiento  1728 non-null   object
 2   puertas               1728 non-null   object
 3   personas              1728 non-null   object
 4   maletero              1728 non-null   object
 5   seguridad             1728 non-null   object
 6   clase                 1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB
None

--- Descripción rápida ---


,count,unique,top,freq
precio_compra,1728,4,vhigh,432
precio_mantenimiento,1728,4,vhigh,432
puertas,1728,4,2,432
personas,1728,3,2,576
maletero,1728,3,small,576
seguridad,1728,3,low,576
clase,1728,4,unacc,1210



--- Distribución de la variable objetivo "class" ---


KeyError: 'class'

In [38]:
# @title
# Renombrar las columnas a español para mayor claridad
df.rename(columns={
    'buying': 'precio_compra',
    'maint': 'precio_mantenimiento',
    'doors': 'puertas',
    'persons': 'personas',
    'lug_boot': 'maletero',
    'safety': 'seguridad',
    'class': 'clase'
}, inplace=True)

# Mostrar las primeras filas del DataFrame con los nombres de columnas actualizados
display(df.head())

,precio_compra,precio_mantenimiento,puertas,personas,maletero,seguridad,clase
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [39]:
# @title
import plotly.express as px

# Gráfico de distribución por clase
fig = px.histogram(df, x='clase',
                   color='clase',
                   title='🚗 Distribución de Clases de Autos',
                   color_discrete_sequence=px.colors.sequential.Tealgrn,
                   labels={'clase':'Clase de Auto'},
                   text_auto=True)  # muestra los conteos sobre las barras

# Estilo automotriz
fig.update_layout(
    title_font_size=24,
    title_font_color='darkred',
    xaxis_title='Clase',
    yaxis_title='Cantidad de Autos',
    plot_bgcolor='lightgray',  # fondo gris tipo pista
    paper_bgcolor='whitesmoke',
    font=dict(family='Arial', size=14, color='black'),
    showlegend=False
)

fig.show()


In [47]:
# @title
import plotly.express as px

# Columnas categóricas a graficar
categorical_cols = ['precio_compra', 'precio_mantenimiento', 'seguridad']

for c in categorical_cols:
    fig = px.histogram(df, x=c,
                       color=c,
                       title=f'🚗 Distribución: {c}',
                       color_discrete_sequence=px.colors.sequential.Tealgrn,
                       text_auto=True,
                       category_orders={c: sorted(df[c].unique())})

    fig.update_layout(
        title_font_size=20,
        title_font_color='darkred',
        xaxis_title=c.replace('_', ' ').capitalize(),
        yaxis_title='Cantidad de Autos',
        plot_bgcolor='lightgray',
        paper_bgcolor='whitesmoke',
        font=dict(family='Arial', size=14, color='black'),
        showlegend=False
    )

    fig.show()


In [49]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 25px;
    background: rgba(0, 0, 30, 0.85);
    border-radius: 20px;
    box-shadow: 0 0 35px #ffcc00;
    width: 85%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h2 style="
        text-align: center;
        font-size: 28px;
        color: #ffcc00;
        text-shadow: 0 0 12px #ffcc00;
    ">📊 Visualización y Análisis Exploratorio</h2>
    <p style="font-size:18px; line-height:1.6;">
        Se transformaron todas las variables al español y se generaron gráficos interactivos.
        En primera instancia, se observa la distribución de los autos según las clases,
        y posteriormente se comparan algunas variables categóricas relevantes.
    </p>
</div>
""")


# Armado de modelo

##Preparar datos para clasificación

In [46]:
# X = features, y = clase
X = pd.get_dummies(df.drop(columns=['clase']))  # codificación rápida
y = df['clase']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)


#Modelo 1: Random KNN


In [41]:
# Escalar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

y_pred_knn = knn.predict(X_test_scaled)
print('KNN Accuracy:', accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))


KNN Accuracy: 0.8634259259259259
              precision    recall  f1-score   support

         acc       0.79      0.71      0.75        96
        good       0.56      0.53      0.55        17
       unacc       0.90      0.97      0.94       303
       vgood       0.50      0.06      0.11        16

    accuracy                           0.86       432
   macro avg       0.69      0.57      0.58       432
weighted avg       0.85      0.86      0.85       432



#Modelo 2: Random Forest

In [42]:
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
print('Random Forest Accuracy:', accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.9699074074074074
              precision    recall  f1-score   support

         acc       0.91      0.97      0.94        96
        good       1.00      0.88      0.94        17
       unacc       0.99      0.98      0.99       303
       vgood       0.93      0.88      0.90        16

    accuracy                           0.97       432
   macro avg       0.96      0.93      0.94       432
weighted avg       0.97      0.97      0.97       432



#Modelo 3: Stacking Classifier

In [43]:
stack = StackingClassifier(
    estimators=[
        ('knn', KNeighborsClassifier(n_neighbors=5)),
        ('rf', RandomForestClassifier(n_estimators=150, random_state=42)),
        ('lr', LogisticRegression(max_iter=1000))
    ],
    final_estimator=LogisticRegression(max_iter=1000),
    cv=5,
    n_jobs=-1
)

stack.fit(X_train_scaled, y_train)
y_pred_stack = stack.predict(X_test_scaled)

print('Stacking Accuracy:', accuracy_score(y_test, y_pred_stack))
print(classification_report(y_test, y_pred_stack))


Stacking Accuracy: 0.9675925925925926
              precision    recall  f1-score   support

         acc       0.93      0.94      0.93        96
        good       1.00      0.94      0.97        17
       unacc       0.98      0.98      0.98       303
       vgood       0.89      1.00      0.94        16

    accuracy                           0.97       432
   macro avg       0.95      0.96      0.96       432
weighted avg       0.97      0.97      0.97       432



#Comparación de modelos

In [51]:
# @title
import plotly.express as px
import pandas as pd

# Resultados de los modelos
results = {
    'Modelo': ['KNN', 'RandomForest', 'Stacking'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_knn),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_stack)
    ],
    'F1_macro': [
        f1_score(y_test, y_pred_knn, average='macro'),
        f1_score(y_test, y_pred_rf, average='macro'),
        f1_score(y_test, y_pred_stack, average='macro')
    ]
}

res_df = pd.DataFrame(results)
res_df


,Modelo,Accuracy,F1_macro
0,KNN,0.863426,0.584711
1,RandomForest,0.969907,0.941299
2,Stacking,0.967593,0.955912


In [52]:
# @title
fig = px.bar(res_df.melt(id_vars='Modelo', value_vars=['Accuracy','F1_macro']),
             x='Modelo', y='value', color='variable',
             barmode='group',
             text='value',
             labels={'value':'Valor', 'variable':'Métrica'},
             title='🚗 Comparación interactiva de modelos')

fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.update_layout(template='plotly_dark', yaxis=dict(range=[0,1]))
fig.show()


In [53]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 25px;
    background: rgba(0, 0, 30, 0.85);
    border-radius: 20px;
    box-shadow: 0 0 40px #ff9900;
    width: 85%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h2 style="
        text-align: center;
        font-size: 28px;
        color: #ff9900;
        text-shadow: 0 0 12px #ff9900;
    ">📊 Resumen de Resultados de Modelos</h2>
    <p style="font-size:18px; line-height:1.6;">
        Análisis breve de los resultados obtenidos para los tres modelos de clasificación:
    </p>
    <ul style="font-size:17px; line-height:1.6;">
        <li><b>KNN:</b> Accuracy ≈ 0.863, F1_macro ≈ 0.585. Desempeño correcto, pero con dificultades para balancear clases minoritarias.</li>
        <li><b>Random Forest:</b> Accuracy ≈ 0.970, F1_macro ≈ 0.941. Excelente desempeño general y alta precisión para la mayoría de clases.</li>
        <li><b>Stacking Classifier:</b> Accuracy ≈ 0.968, F1_macro ≈ 0.956. Combina varios modelos; mejora la medida balanceada (F1_macro), aunque la Accuracy es ligeramente menor que Random Forest.</li>
    </ul>
    <p style="font-size:18px; line-height:1.6;">
        <b>Conclusión:</b> Random Forest tiene la mayor precisión global, mientras que Stacking Classifier ofrece mejor balance entre clases. KNN sirve como referencia simple.
    </p>
</div>
""")


# **Conclusion**

In [54]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 30px;
    background: rgba(0, 0, 30, 0.9);
    border-radius: 20px;
    box-shadow: 0 0 45px #ff4444;
    width: 85%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h2 style="
        text-align: center;
        font-size: 28px;
        color: #ff4444;
        text-shadow: 0 0 12px #ff4444;
    ">🏁 Conclusión Final del TP1 - PP2</h2>
    <p style="font-size:18px; line-height:1.6;">
        Realicé un trabajo explorando datos de automóviles, intentando predecir y diferenciar en 4 clases,
        entrenando distintos modelos de clasificación.
    </p>
    <p style="font-size:18px; line-height:1.6;">
        Como resultado final, se concluyó que el modelo que mejor se adaptó fue <b>Random Forest</b>.
    </p>
    <p style="font-size:18px; line-height:1.6;">
        Una vez creado todo el código y los modelos, se ajustaron los marcadores y estilos para presentar
        un trabajo visualmente más atractivo, utilizando la ayuda de ChatGPT para generar HTML y displays interactivos.
    </p>
    <p style="font-size:18px; line-height:1.6;">
        Para próximas entregas se planea realizar una valorización del modelo, intentando predecir cuál es la mejor opción
        en términos prácticos y económicos.
    </p>
</div>
""")



#HASTA ACÁ VA EL TP1

In [ ]:
# Crear una lista con las etiquetas de las características
feature_labels = ['exi', 'psi', 'experiencia_laboral', 'edad']

featureImportances = pd.Series(clf.feature_importances_).sort_values(ascending=False)
print(featureImportances)

# Crear un gráfico de barras utilizando Seaborn con etiquetas personalizadas
ax = sn.barplot(x=round(featureImportances, 4), y=featureImportances.index, orient='h')

# Configurar las etiquetas de las características en la gráfica
ax.set_yticklabels(feature_labels)

plt.show()

#DESDE ACA DEBE TERMINAR TP2_ CON LA PARTE ECNÓMICA + conclusiones interactivas !!!

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Después de realizar la predicción
def interpret_prediction(prediction):
    if prediction == 2:
        return "Clase 'Candidato admitido'"
    elif prediction == 1:
        return "Clase 'Candidato en lista de espera'"
    elif prediction == 0:
        return "Clase 'Candidato no admitido o rechazado'"
    else:
        return "Resultado no reconocido"

In [ ]:
def predict(exi, psi, experiencia_laboral, edad):
    prediction = clf.predict([[exi, psi, experiencia_laboral, edad]])
    return prediction[0]

exi_slider = widgets.FloatSlider(value=500, min=0, max=1000, step=10, description='Calificación Examen de Ingreso:')
psi_slider = widgets.FloatSlider(value=30, min=0, max=100, step=1, description='Puntaje Psicofísico:')
experiencia_slider = widgets.FloatSlider(value=5, min=0, max=10, step=1, description='Experiencia Laboral:')
edad_slider = widgets.FloatSlider(value=40, min=0, max=100, step=1, description='Edad:')

output = widgets.Output()

In [ ]:
def on_predict_button_clicked(b):
    with output:
        prediction = predict(exi_slider.value, psi_slider.value, experiencia_slider.value, edad_slider.value)
        interpretation = interpret_prediction(prediction)
        print(f"El modelo predice: {interpretation}")

predict_button = widgets.Button(description='Realizar Predicción')
predict_button.on_click(on_predict_button_clicked)

display(exi_slider, psi_slider, experiencia_slider, edad_slider, predict_button, output)

##El objetivo es realizar una valoración económica basada en la precisión del modelo, utilizando la matriz de confusión para determinar el impacto económico de las predicciones correctas e incorrectas y ver como tomar decisiones acertadas basadas en datos en el dpto de rrhh

In [ ]:
import numpy as np  # Importa la biblioteca numpy para manejar arreglos numéricos
import pandas as pd  # Importa la biblioteca pandas para manipular datos en forma de DataFrames

# Definir los valores económicos para cada clase
economic_values_correct = [20000, 50000, 100000]  # Valores asociados a las predicciones correctas para [Clase 0, Clase 1, Clase 2]
penalty_incorrect = -50000  # Penalización aplicada a las predicciones incorrectas

# Crear la matriz de confusión
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
# Calcula la matriz de confusión comparando las predicciones del modelo (y_pred) con los valores reales (y_test)

# Calcular el impacto económico total
economic_impact = 0  # Inicializa la variable que acumulará el impacto económico total

# Recorrer la matriz de confusión
for i in range(len(confusion_matrix)):
    for j in range(len(confusion_matrix)):
        if i == j:
            # Predicciones correctas
            economic_impact += confusion_matrix.iloc[i, j] * economic_values_correct[i]
            # Suma el valor económico correspondiente a las predicciones correctas (diagonal de la matriz)
        else:
            # Predicciones incorrectas
            economic_impact += confusion_matrix.iloc[i, j] * penalty_incorrect
            # Suma la penalización por las predicciones incorrectas (fuera de la diagonal)

# Mostrar el impacto económico total
print(f"Impacto Económico Total: ${economic_impact:,.2f}")  # Imprime el impacto económico total en un formato legible


## Implementación de una SVM a nivel empresa / retomando caso RRHH para ver cual seria el mejor modelo para este caso :)

In [ ]:
# Importar librerías necesarias
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import plotly.express as px

# Aquí estamos utilizando el clasificador SVM en lugar de Random Forest
clf = SVC(kernel='linear')  # Puedes cambiar 'linear' por 'rbf', 'poly', etc., dependiendo del kernel que prefieras
clf.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Definir las etiquetas de las clases
class_labels = ["No admitido", "En lista de espera", "Admitido"]

# Crear la matriz de confusión
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])

# Crear una figura interactiva con Plotly Express
fig = px.imshow(confusion_matrix,
                labels=dict(x="Predicted", y="Actual", color="Count"),
                x=class_labels,  # Usa las etiquetas de las clases para el eje x
                y=class_labels,  # Usa las etiquetas de las clases para el eje y
                color_continuous_scale="Viridis")

# Configurar los ejes
fig.update_xaxes(side="top")  # Muestra la leyenda de la matriz en la parte superior
fig.update_layout(
    xaxis=dict(categoryorder='array', categoryarray=class_labels),  # Ordena las categorías en el eje x
    yaxis=dict(categoryorder='array', categoryarray=class_labels)   # Ordena las categorías en el eje y
)

# Mostrar la figura interactiva
fig.show()

# Evaluar el rendimiento
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo SVM: {accuracy * 100:.2f}%')


## Evaluación

In [ ]:
import numpy as np  # Importa la biblioteca numpy para manejar arreglos numéricos
import pandas as pd  # Importa la biblioteca pandas para manipular datos en forma de DataFrames

# Definir los valores económicos para cada clase
economic_values_correct = [20000, 50000, 100000]  # Valores asociados a las predicciones correctas para [Clase 0, Clase 1, Clase 2]
penalty_incorrect = -50000  # Penalización aplicada a las predicciones incorrectas

# Crear la matriz de confusión
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
# Calcula la matriz de confusión comparando las predicciones del modelo (y_pred) con los valores reales (y_test)

# Calcular el impacto económico total
economic_impact = 0  # Inicializa la variable que acumulará el impacto económico total

# Recorrer la matriz de confusión
for i in range(len(confusion_matrix)):
    for j in range(len(confusion_matrix)):
        if i == j:
            # Predicciones correctas
            economic_impact += confusion_matrix.iloc[i, j] * economic_values_correct[i]
            # Suma el valor económico correspondiente a las predicciones correctas (diagonal de la matriz)
        else:
            # Predicciones incorrectas
            economic_impact += confusion_matrix.iloc[i, j] * penalty_incorrect
            # Suma la penalización por las predicciones incorrectas (fuera de la diagonal)

# Mostrar el impacto económico total
print(f"Impacto Económico Total: ${economic_impact:,.2f}")  # Imprime el impacto económico total en un formato legible


In [ ]:
## En comparación con forest gana la maquina a nivel empresarial para este caso

In [ ]:
# conclusiones generales de PF